In [16]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

if device.type=='cuda':
    print(torch.cuda.get_device_name(0))

cpu


In [17]:
CLASSES=10
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [18]:
import torch
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
#import matplotlib.pyplot as plt
import numpy as np

In [19]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1,out_channels=96,kernel_size=11,stride=4,padding=0) 
        #Layer1: 1. Input will be 1 channel i.e grayscale images 2. Apply 96 kernels, of kernel size 11, stride of 4 and padding of 0 3. Apply activation function ReLU

        self.conv2 = nn.Conv2d(in_channels=96,out_channels=96,kernel_size=1)   
        #Layer2:1. Apply 96 kernels, kernel size 1 2. Apply activation function ReLU

        self.conv3 = nn.Conv2d(in_channels=96,out_channels=96,kernel_size=1)      
        self.local_response1=nn.LocalResponseNorm(size=5,alpha=0.0001,beta=0.75,k=2)
        self.pool1 = nn.MaxPool2d(kernel_size=3,stride=2)
        #Layer3: 1. Apply 96 kernel, kernel size 1 2. Apply activation function ReLU 3. Do a maxpool2d, kernel size of 3 and stride2 4. Dropout of 0.5

        self.conv4=nn.Conv2d(in_channels=96,out_channels=256,kernel_size=11,stride=4,padding=2)
        #Layer4: 1. Apply 256 kernel (hint the input to this conv2d layer is still 96) of size 11, stride 4 and padding 2 2. Apply activation function ReLU

        self.conv5=nn.Conv2d(in_channels=256,out_channels=256,kernel_size=1) 
        #Layer5: 1. Apply 256 kernels, kernel size of 1 2. Apply activation function ReLU   (there is 2 layer 5, so actually 12 layers in total)

        self.conv6=nn.Conv2d(in_channels=256,out_channels=256,kernel_size=1) 
        self.local_response1=nn.LocalResponseNorm(size=5,alpha=0.0001,beta=0.75,k=2)
        self.pool2 = nn.MaxPool2d(kernel_size=3,stride=2)
        #Layer5: 1. Apply 256 kernels, kernel size of 1 2. Apply activation function ReLU 3. Maxpool2d, kernel size of 3 and stride of 2 4. Dropout of 0.5    (there is 2 layer 5, so actually 12 layers in total)

        self.conv7=nn.Conv2d(in_channels=256,out_channels=384,kernel_size=3,stride=1,padding=1)
        #Layer6: 1. Apply 384 kernels of size 3, stride of 1 and padding of 1  2. Apply activation function ReLU

        self.conv8=nn.Conv2d(in_channels=384,out_channels=384,kernel_size=1)
        #Layer7:1. Apply 384 kernels of size 1 2. Apply activation function ReLU

        self.conv9=nn.Conv2d(in_channels=384,out_channels=384,kernel_size=1)
        #Layer 8: 1. Apply 384 kernels of size 1 2. Apply activation function ReLU 3. Dropout of 0.5

        self.conv10=nn.Conv2d(in_channels=384,out_channels=10,kernel_size=3,stride=1,padding=1)
        #Layer 9: 1. Apply 10 kernel (hint the input to this layer is 384) of kernel size 3, stride of 1 andpadding of 1 2. Apply activation function ReLU

        self.conv11=nn.Conv2d(in_channels=10,out_channels=10,kernel_size=1)
        #Layer 10: 1. Apply 10 kernels of kernel size 1 2. Apply activation function ReLU

        self.conv12=nn.Conv2d(in_channels=10,out_channels=10,kernel_size=1)
        #Layer 11: 1. Apply 10 kernels of kernel size 1 2. Apply activation function ReLU 3. Finally apply nn.AdaptiveAvgPool2d((1, 1)) to the final output.
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        #Layer1: 1. Input will be 1 channel i.e grayscale images 2. Apply 96 kernels, of kernel size 11, stride of 4 and padding of 3. Apply activation function ReLU

        x = F.relu(self.conv2(x))
        #Layer2: 1. Apply 96 kernels, kernel size 1 2. Apply activation function ReLU

        x = self.pool1(F.relu(self.local_response1(self.conv3(x)))) 
        x = F.dropout(x, 0.5)
        #Layer3: 1. Apply 96 kernel, kernel size 1 2. Apply activation function ReLU 3. Do a maxpool2d, kernel size of 3 and stride2 4. Dropout of 0.5

        x = F.relu(self.conv4(x))
        #Layer4: 1. Apply 256 kernel (hint the input to this conv2d layer is still 96) of size 11, stride 4 and padding 2 2. Apply activation function ReLU

        x = F.relu(self.conv5(x))
        #Layer5: 1. Apply 256 kernels, kernel size of 1 2. Apply activation function ReLU

        x = self.pool1(F.relu(self.local_response1(self.conv6(x))))
        x = F.dropout(x, 0.5)
        #Layer5: 1. Apply 256 kernels, kernel size of 1 2. Apply activation function ReLU 3. Maxpool2d, kernel size of 3 and stride of 2. 4. Dropout of 0.5


        x = F.relu(self.conv7(x))
        #Layer6: 1. Apply 384 kernels of size 3, stride of 1 and padding of 1 2. Apply activation function ReLU

        x = F.relu(self.conv8(x))
        #Layer7:1. Apply 384 kernels of size 1 2. Apply activation function ReLU


        x = F.relu(self.conv9(x))
        x = F.dropout(x, 0.5)
        #Layer 8: 1. Apply 384 kernels of size 1 2. Apply activation function ReLU 3. Dropout of 0.5

        x = F.relu(self.conv10(x))
        #Layer 9: 1. Apply 10 kernel (hint the input to this layer is 384) of kernel size 3, stride of 1 andpadding of 1 2. Apply activation function ReLU

        x = F.relu(self.conv11(x))
        #Layer 10: 1. Apply 10 kernels of kernel size 1 2. Apply activation function ReLU

        x = F.relu(self.conv12(x))
        #Layer 11: 1. Apply 10 kernels of kernel size 1 2. Apply activation function ReLU 3. Finally apply nn.AdaptiveAvgPool2d((1, 1)) to the final output.
        #print("forward done")
        x = self.avg_pool(self.conv12(x))
        x = torch.flatten(x,1)
        return x


In [20]:
transform_conf=transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,))
])

In [21]:
BATCH_SIZE=64
train_dataset=datasets.MNIST('/users/jizhang',train=True,download=True,transform=transform_conf,)
test_dataset=datasets.MNIST('/users/jizhang',train=False,download=True,transform=transform_conf)

In [22]:
train_loader=torch.utils.data.DataLoader(train_dataset,batch_size=BATCH_SIZE,shuffle=True)
test_loader=torch.utils.data.DataLoader(test_dataset,batch_size=BATCH_SIZE,shuffle=True)

In [23]:
model=Net().to(device)
optimizer=optim.Adam(params=model.parameters(),lr=0.0001)
loss_fn = nn.CrossEntropyLoss()

In [24]:
def train(model,device,train_loader,optimizer,epochs):
    print("inside train")
    model.train()
    for batch_ids, (img, classes) in enumerate(train_loader):
        classes=classes.type(torch.LongTensor)
        img,classes=img.to(device),classes.to(device)
        torch.autograd.set_detect_anomaly(True)     
        optimizer.zero_grad()
        output=model(img)
        loss = loss_fn(output,classes)                
        
        loss.backward()
        optimizer.step()
    if(batch_ids +1) % 2 == 0:
        print("Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
            epoch, batch_ids* len(img), len(train_loader.dataset),
            100.*batch_ids / len(train_loader),loss.item()))

In [25]:
def test(model, device, test_loader):
    model.eval()
    test_loss=0
    correct=0
    with torch.no_grad():
        for img,classes in test_loader:
            img,classes=img.to(device), classes.to(device)
            y_hat=model(img)
            test_loss+=F.nll_loss(y_hat,classes,reduction='sum').item()
            _,y_pred=torch.max(y_hat,1)
            correct+=(y_pred==classes).sum().item()
        test_loss/=len(test_dataset)
        print("\n Test set: Avarage loss: {:.0f},Accuracy:{}/{} ({:.0f}%)\n".format(
            test_loss,correct,len(test_dataset),100.*correct/len(test_dataset)))
        print('='*30)

In [26]:
if __name__=='__main__':
    seed=42
    EPOCHS=1
    
    for epoch in range(1,EPOCHS+1):
        train(model,device,train_loader,optimizer,epoch)
        test(model,device,test_loader)

inside train
Train Epoch: 1 [29984/60000 (100%)]	Loss: 0.752769

 Test set: Avarage loss: -6,Accuracy:7807/10000 (78%)

